In [1]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import netCDF4 as nc
import geopandas as gpd
import rioxarray as rxr
from myfunc import timer
from myfunc import DirMan
from myfunc import load_and_flatten_data
import config

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# configuration
# resolution = "0p1"
resolution = "500"
region = [-180,180,-60,90]
data_path = f'/tera04/zhwei/xionghui/bedrock/run/{resolution}/'
post_data_path = '/tera04/zhwei/xionghui/bedrock/'
shp_path = '/tera04/zhwei/xionghui/bedrock/Shp/'
fig_path = f'/home/xuxh22/stu01/Bedrock/fig/{resolution}/'
path = '/home/xuxh22/stu01/Bedrock/'
if resolution == "0p1":
    size = 0.1
elif resolution == "500":
    size = 0.0005

dir_man = DirMan(data_path)
dir_man.enter()

os.makedirs(f'{data_path}/csv', exist_ok=True)

Entered /tera04/zhwei/xionghui/bedrock/run/500/


In [2]:
df = pd.read_csv(f'{post_data_path}/field/field_all.csv', encoding='latin-1')
print(post_data_path)


# dir_man = DirMan(data_path)
# dir_man.enter()

s1 = nc.Dataset('Sbedrock_tmp2.nc4', 'r')
s2 = nc.Dataset('Ssoil.nc4', 'r')
s3 = nc.Dataset('DTB.nc4', 'r')
s4 = nc.Dataset('mask1.nc4', 'r')
s5 = nc.Dataset('mask2.nc4', 'r')
s6 = nc.Dataset('mask3.nc4', 'r')
s7 = nc.Dataset('mask4.nc4', 'r')
s8 = nc.Dataset('mask1234.nc4', 'r')

lat = df['Latitude']
lon = df['Longitude']
ssa = df['Same Site As']
# print(lat,lon)

lat1 = s1.variables['lat'][:]
lon1 = s1.variables['lon'][:]
# print(lat1,lon1)

df2 = pd.DataFrame()
j = 0
for i in range(len(lat)):
    # if not isinstance(ssa[i], str):
        # if 
        # print('-----------------------------------------------')
        
    lat1_index = np.argmin(np.abs(lat1 - lat[i]))
    lon1_index = np.argmin(np.abs(lon1 - lon[i]))
    lat1_target = lat1[lat1_index]
    lon1_target = lon1[lon1_index]
    # print(lat[i])
    # print(lon[i])
    # print('find ')
    # print(lat1_target)
    # print(lon1_target)
    # print('-----------------------------------------------')
    
    df2.loc[j, 'Measure'] = df.loc[i, 'Measurement or Estimate of RM Contribution to ET?']
    df2.loc[j, 'Citation'] = df.loc[i, 'Citation']
    df2.loc[j, 'lat'] = round(lat[i], 6)
    df2.loc[j, 'lon'] = round(lon[i], 6)
    df2.loc[j, 'lat'] = lat[i]
    df2.loc[j, 'lon'] = lon[i]
    df2.loc[j, 'Sbedrock_field_min'] = df.loc[i, 'Minimum']
    df2.loc[j, 'Sbedrock_field_max'] = df.loc[i, 'Maximum']
    df2.loc[j, 'Sbedrock'] = s1['Sbedrock'][lat1_index,lon1_index]
    df2.loc[j, 'Ssoil'] = s2['Band1'][lat1_index,lon1_index]
    df2.loc[j, 'Soil_depth'] = df.loc[i, 'SoilDepth_Numberline_cm']
    df2.loc[j, 'DTB'] = s3['Band1'][lat1_index,lon1_index]
    df2.loc[j, 'mask1'] = s4['Band1'][lat1_index,lon1_index]
    df2.loc[j, 'mask2'] = s5['LC'][0,lat1_index,lon1_index]
    df2.loc[j, 'mask3'] = s6['et'][0,lat1_index,lon1_index]
    df2.loc[j, 'mask4'] = s7['Band1'][lat1_index,lon1_index]
    df2.loc[j, 'mask'] = s8['Band1'][lat1_index,lon1_index]
    
    j += 1

unique_pairs = df2[['lat', 'lon']].drop_duplicates().shape[0]
print(f"Unique (lat, lon) pairs in df2: {unique_pairs}")

df3 = df2.groupby(['lat', 'lon']).first().reset_index()
# df3.fillna(0, inplace=True)
df3['num'] = range(len(df3['lat']))
with open('csv/site.csv','w') as f:
    df3.to_csv(f)
    
print((df3['Measure']=='N').sum())
print((df3['Measure']=='Y').sum())
df3
# df

/tera04/zhwei/xionghui/bedrock/


/home/xuxh22/anaconda3/envs/jupyter/lib/python3.10/site-packages/pandas/core/internals/blocks.py:1429: UserWarning: Warning: converting a masked element to nan.
  values[indexer] = casted
/home/xuxh22/anaconda3/envs/jupyter/lib/python3.10/site-packages/pandas/core/internals/blocks.py:1429: UserWarning: Warning: converting a masked element to nan.
  values[indexer] = casted
/home/xuxh22/anaconda3/envs/jupyter/lib/python3.10/site-packages/pandas/core/internals/blocks.py:1429: UserWarning: Warning: converting a masked element to nan.
  values[indexer] = casted
/home/xuxh22/anaconda3/envs/jupyter/lib/python3.10/site-packages/pandas/core/internals/blocks.py:1429: UserWarning: Warning: converting a masked element to nan.
  values[indexer] = casted
/home/xuxh22/anaconda3/envs/jupyter/lib/python3.10/site-packages/pandas/core/internals/blocks.py:1429: UserWarning: Warning: converting a masked element to nan.
  values[indexer] = casted
/home/xuxh22/anaconda3/envs/jupyter/lib/python3.10/site-pack

Unique (lat, lon) pairs in df2: 80
40
40


/home/xuxh22/anaconda3/envs/jupyter/lib/python3.10/site-packages/pandas/core/internals/blocks.py:1429: UserWarning: Warning: converting a masked element to nan.
  values[indexer] = casted
/home/xuxh22/anaconda3/envs/jupyter/lib/python3.10/site-packages/pandas/core/internals/blocks.py:1429: UserWarning: Warning: converting a masked element to nan.
  values[indexer] = casted


,lat,lon,Measure,Citation,Sbedrock_field_min,Sbedrock_field_max,Sbedrock,Ssoil,Soil_depth,DTB,mask1,mask2,mask3,mask4,mask,num
0,-34.300000,19.150000,N,"Lamont, 1984",NaN,NaN,147.998795,207.693985,60.0,599.061157,NaN,1.0,NaN,NaN,NaN,0
1,-32.338300,116.090400,N,"Dell, 1983",NaN,NaN,300.742432,75.005013,NaN,100.000000,1.0,1.0,NaN,1.0,NaN,1
2,-30.412800,18.249700,N,"Carrick, 2003",NaN,NaN,67.932983,69.475952,0.3,100.000000,1.0,1.0,NaN,NaN,NaN,2
3,11.730000,76.450000,Y,"Ruiz, 2010",NaN,NaN,199.571167,98.559509,NaN,100.000000,1.0,1.0,1.0,1.0,1.0,3
4,20.815556,-89.245000,Y,Querejeta 2007,NaN,NaN,160.841095,233.316879,30.0,5000.000000,NaN,0.0,NaN,NaN,NaN,4
5,20.833333,-89.652778,Y,Querejeta 2007,NaN,NaN,122.585724,231.535812,30.0,5000.000000,NaN,1.0,NaN,NaN,NaN,5
6,20.879167,-89.653333,Y,Querejeta 2007,NaN,NaN,89.471489,228.718231,30.0,5000.000000,NaN,1.0,NaN,NaN,NaN,6
7,20.905000,-89.630000,N,"Estrada-Medina, 2013",NaN,NaN,78.262108,211.198639,15.0,5000.000000,NaN,0.0,NaN,NaN,NaN,7
8,22.000000,-101.000000,N,"Rodriguez-Robles, 2017",NaN,NaN,154.464813,90.615860,NaN,100.000000,1.0,1.0,1.0,NaN,NaN,8
9,24.716700,108.300000,N,"Nie, 2014",NaN,NaN,54.620438,98.653503,30.0,100.000000,1.0,1.0,1.0,NaN,NaN,9


In [ ]:
file_path1 = f'{post_data_path}DTB/DTB_Shangguan/BDTICM_M_1km_ll.tif'
file_path2 = f'{post_data_path}US/products_used/gNATSGO/densic_bedrock_depth_90m.tiff'
file_path3 = f'{post_data_path}US/products_used/gNATSGO/lithic_bedrock_depth_90m.tiff'
file_path4 = f'{post_data_path}US/products_used/gNATSGO/paralithic_bedrock_depth_90m.tiff'


# file_path5 = f'{post_data_path}mask1/mask1_v1/DTB_temp3.nc'
file_path6 = f'{post_data_path}/DTB/DTB_Pelletier/DTB_temp1.nc'
csv_file1 = f'{post_data_path}field/field_all.csv'
csv_file2 = f'{data_path}csv/DTB_US.csv'

s1 = rxr.open_rasterio(file_path1)
s2 = rxr.open_rasterio(file_path2)
s3 = rxr.open_rasterio(file_path3)
s4 = rxr.open_rasterio(file_path4)
s5 = xr.open_dataset(file_path5)
s6 = xr.open_dataset(file_path6)
csv1 = pd.read_csv(csv_file1, encoding='latin-1')

lat2 = csv1['Latitude']
lon2 = csv1['Longitude']
sd = csv1['SoilDepth_Numberline_cm']
ssa = csv1['Same Site As']

# Set the DTB.csv
csv2 = {
    'number':[],
    'lat':[],
    'lon':[],
    'Field':[],
    'gNATSGO':[],
    'SoilGrids250m':[],
    'SoilGrids250m_rev':[],
    'Pelletier':[]
}
csv2 = pd.DataFrame(csv2)

a1 = csv2['gNATSGO']
a2 = csv2['SoilGrids250m']
a3 = csv2['SoilGrids250m_rev']
a4 = csv2['Pelletier']

# Renew the csv data in field
j = 0
for i in range(len(sd)):
    if i!=(len(sd)-1):
        if sd[i] > 0 and not isinstance(ssa[i], str) and lat2[i] != lat2[i + 1] and lon2[i] <= -66.9 and lon2[i] >= -125 and lat2[i] >= 24.4 and lat2[i] <= 49.4:
            csv2.at[j, 'Field'] = sd[i]
            csv2.at[j, 'lat'] = lat2[i]
            csv2.at[j, 'lon'] = lon2[i]
            csv2.at[j, 'number'] = int(j + 1)
            j += 1
            # print(j)
    else:
        if sd[i] > 0 and not isinstance(ssa[i], str) and lat2[i] != lat2[i - 1] and lon2[i] <= -66.9 and lon2[i] >= -125 and lat2[i] >= 24.4 and lat2[i] <= 49.4:
            csv2.at[j, 'Field'] = sd[i]
            csv2.at[j, 'lat'] = lat2[i]
            csv2.at[j, 'lon'] = lon2[i]
            csv2.at[j, 'number'] = int(j + 1)
            j += 1
            # print(j)

print(j)
    

# Index the corresponding location points in global data
lat = csv2['lat']
lon = csv2['lon']
for i in range(j):
    
    lat1_index = abs(s1.coords['y'] - lat[i]).argmin()
    lon1_index = abs(s1.coords['x'] - lon[i]).argmin()

    lat2_index = abs(s2.coords['y'] - lat[i]).argmin()
    lon2_index = abs(s2.coords['x'] - lon[i]).argmin()
    lat3_index = abs(s3.coords['y'] - lat[i]).argmin()
    lon3_index = abs(s3.coords['x'] - lon[i]).argmin()
    lat4_index = abs(s4.coords['y'] - lat[i]).argmin()
    lon4_index = abs(s4.coords['x'] - lon[i]).argmin()

    lat5_index = abs(s5.coords['lat'] - lat[i]).argmin()
    lon5_index = abs(s5.coords['lon'] - lon[i]).argmin()
    lat6_index = abs(s6.coords['lat'] - lat[i]).argmin()
    lon6_index = abs(s6.coords['lon'] - lon[i]).argmin()

    a1_value = s1[0, lat1_index, lon1_index].values

    a2_value = s2[0, lat2_index, lon2_index].values
    a3_value = s3[0, lat3_index, lon3_index].values
    a4_value = s4[0, lat4_index, lon4_index].values

    # a5_value = s5['Band1'][lat5_index, lon5_index].values
    a6_value = s6['Band1'][lat6_index, lon6_index].values

    a1[i] = a1_value
    print(abs(s2.coords['y'] - lat[i]).argmin())
    # if abs(s2.coords['y'] - lat[i]).argmin() > 5:
    #     a2[i] = 0
    # else:
    a2[i] = np.min([a2_value,a3_value,a4_value])


    # a3[i] = a5_value
    a4[i] = a6_value

csv2['SoilGrids250m'] = a1
csv2['gNATSGO'] = a2
# csv2['SoilGrids250m_rev'] = a3
csv2['Pelletier'] = a4

csv2.to_csv(csv_file2, index=False)

print("the new DTB.csv is:")
# print(csv2)
csv2

In [ ]:
file_path1 = f'{post_data_path}mask1/mask1_v1/BDTICM_M_1km_ll.tif'
file_path2 = f'{post_data_path}US/products_used/gNATSGO/densic_bedrock_depth_90m.tiff'
file_path3 = f'{post_data_path}US/products_used/gNATSGO/lithic_bedrock_depth_90m.tiff'
file_path4 = f'{post_data_path}US/products_used/gNATSGO/paralithic_bedrock_depth_90m.tiff'


file_path5 = f'{post_data_path}mask1/mask1_v1/DTB_temp3.nc'
file_path6 = f'{post_data_path}mask1/mask1_v3/DTB_temp1.nc'
csv_file1 = f'{post_data_path}field/field_all.csv'
csv_file2 = f'{data_path}csv/DTB_G.csv'

s1 = rxr.open_rasterio(file_path1)
s2 = rxr.open_rasterio(file_path2)
s3 = rxr.open_rasterio(file_path3)
s4 = rxr.open_rasterio(file_path4)
s5 = xr.open_dataset(file_path5)
s6 = xr.open_dataset(file_path6)
csv1 = pd.read_csv(csv_file1, encoding='latin-1')

lat2 = csv1['Latitude']
lon2 = csv1['Longitude']
sd = csv1['SoilDepth_Numberline_cm']
ssa = csv1['Same Site As']

# Set the DTB.csv
csv2 = {
    'number':[],
    'lat':[],
    'lon':[],
    'Field':[],
    'gNATSGO':[],
    'SoilGrids250m':[],
    'SoilGrids250m_rev':[],
    'Pelletier':[]
}
csv2 = pd.DataFrame(csv2)

a1 = csv2['gNATSGO']
a2 = csv2['SoilGrids250m']
a3 = csv2['SoilGrids250m_rev']
a4 = csv2['Pelletier']

# Renew the csv data in field
j = 0
for i in range(len(sd)):
    if i!=(len(sd)-1):
        if sd[i] > 0 and not isinstance(ssa[i], str) and lat2[i] != lat2[i + 1]:
            csv2.at[j, 'Field'] = sd[i]
            csv2.at[j, 'lat'] = lat2[i]
            csv2.at[j, 'lon'] = lon2[i]
            csv2.at[j, 'number'] = int(j + 1)
            j += 1
    else:
        if sd[i] > 0 and not isinstance(ssa[i], str) and lat2[i] != lat2[i - 1]:
            csv2.at[j, 'Field'] = sd[i]
            csv2.at[j, 'lat'] = lat2[i]
            csv2.at[j, 'lon'] = lon2[i]
            csv2.at[j, 'number'] = int(j + 1)
            j += 1

print(j)

# Index the corresponding location points in global data
lat = csv2['lat']
lon = csv2['lon']
for i in range(j):
    lat1_index = abs(s1.coords['y'] - lat[i]).argmin()
    lon1_index = abs(s1.coords['x'] - lon[i]).argmin()

    lat2_index = abs(s2.coords['y'] - lat[i]).argmin()
    lon2_index = abs(s2.coords['x'] - lon[i]).argmin()
    lat3_index = abs(s3.coords['y'] - lat[i]).argmin()
    lon3_index = abs(s3.coords['x'] - lon[i]).argmin()
    lat4_index = abs(s4.coords['y'] - lat[i]).argmin()
    lon4_index = abs(s4.coords['x'] - lon[i]).argmin()

    lat5_index = abs(s5.coords['lat'] - lat[i]).argmin()
    lon5_index = abs(s5.coords['lon'] - lon[i]).argmin()
    lat6_index = abs(s6.coords['lat'] - lat[i]).argmin()
    lon6_index = abs(s6.coords['lon'] - lon[i]).argmin()

    a1_value = s1[0, lat1_index, lon1_index].values

    a2_value = s2[0, lat2_index, lon2_index].values
    a3_value = s3[0, lat3_index, lon3_index].values
    a4_value = s4[0, lat4_index, lon4_index].values

    a5_value = s5['Band1'][lat5_index, lon5_index].values
    a6_value = s6['Band1'][lat6_index, lon6_index].values

    a1[i] = a1_value
    print(abs(s2.coords['y'] - lat[i]).argmin())
    if abs(s2.coords['y'] - lat[i]).argmin() > 5:
        a2[i] = 0
    else:
        a2[i] = np.min([a2_value,a3_value,a4_value])


    a3[i] = a5_value
    a4[i] = a6_value

csv2['SoilGrids250m'] = a1
csv2['gNATSGO'] = a2
csv2['SoilGrids250m_rev'] = a3
csv2['Pelletier'] = a4

csv2.to_csv(csv_file2, index=False)

print("the new DTB_G.csv is:")
# print(csv2)
csv2

In [ ]:
file_path1 = f'{data_path}Ssoil.nc4'
file_path2 = f'{data_path}Sbedrock_tmp2.nc4'
csv_file1 = f'{post_data_path}field/field_all.csv'
csv_file2 = f'{data_path}csv/Sbedrock.csv'

s1 = xr.open_dataset(file_path1)
s2 = xr.open_dataset(file_path2)
csv1 = pd.read_csv(csv_file1, encoding='latin-1')

lat2 = csv1['Latitude']
lon2 = csv1['Longitude']
min = csv1['Minimum']
max = csv1['Maximum']
ssa = csv1['Same Site As']
# nf =  csv1['Number_For_Plotting']

# Set the Sbedrock.csv
csv2 = {
    'number':[],
    'lat':[],
    'lon':[],
    'min':[],
    'max':[],
    'Ssoil':[],
    'Sbedrock':[]
}
csv2 = pd.DataFrame(csv2)

a1 = csv2['Ssoil']
a2 = csv2['Sbedrock']

# Renew the csv data in field
j = 0
for i in range(len(lat2)):
    if isinstance(min[i], str) and not isinstance(ssa[i], str):
        csv2.at[j, 'lat'] = lat2[i]
        csv2.at[j, 'lon'] = lon2[i]
        csv2.at[j, 'min'] = min[i]
        csv2.at[j, 'max'] = max[i]
        csv2.at[j, 'number'] = int(j + 1)
        j += 1
        print(j)

# Index the corresponding location points in global data
lat = csv2['lat']
lon = csv2['lon']
for i in range(j):
    lat1_index = abs(s1.coords['lat'] - lat[i]).argmin()
    lon1_index = abs(s1.coords['lon'] - lon[i]).argmin()
    lat2_index = abs(s2['lat'] - lat[i]).argmin()
    lon2_index = abs(s2['lon'] - lon[i]).argmin()

    a1_value = s1['Band1'][lat1_index, lon1_index].values
    a2_value = s2['Sbedrock'][lat2_index, lon2_index].values

    a1[i] = a1_value
    a2[i] = a2_value
    print('the Ssoil is',a1[i])
    print('the Sbedrock is',a2[i])

csv2['Ssoil'] = a1
csv2['Sbedrock'] = a2

csv2.to_csv(csv_file2, index=False)

print("the new Sbedrock.csv is:")
print(csv2)
csv2